<a href="https://colab.research.google.com/github/Ezekiel112/ChexPert-CNN/blob/main/Arrythmia_detection_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!nvidia-smi

Sat Feb 12 17:06:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
shutil.copyfile('/content/drive/MyDrive/Thesis.zip', './arrhythmia_detection_ECG.zip')
!unzip arrhythmia_detection_ECG.zip
#os.remove("arrhythmia_detection_ECG.zip")

In [9]:
import os
os.getcwd()

'/content'

In [1]:
%cd Thesis

/content/Thesis


In [ ]:
shutil.copyfile('/content/drive/MyDrive/training_data.zip', './training_data.zip')
shutil.copyfile('/content/drive/MyDrive/test_data.zip', './test.zip')
!unzip training_data
!unzip test_data
!unzip test.zip

In [2]:
import torch
import torchvision
import torch.nn as nn
import numpy as np, os, sys, joblib
import matplotlib.pyplot as pl
import pandas as pd
import random, os
import librosa
from torch.utils.data.dataset import Dataset
import torch.nn.functional as F
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
 #from torch.utils.data import random_split
from torch.optim import lr_scheduler
import time
import tqdm
from evaluate_model import *
from my_helper_code import *
from helper_code import *
from model import *
from torch.nn import  Conv1d,Conv2d
%matplotlib inline

In [3]:
def seed_everything(seed: int):
    
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(0)

In [4]:
df = pd.read_csv('dx_mapping_scored.csv')
labels = df['SNOMEDCTCode'].values
labels = [str(i) for i in labels]
classes = list(labels)
train_data_directory = 'training_data'
train_header_files, train_recording_files = find_challenge_files(train_data_directory)
train_num_recordings = len(train_recording_files)
twelve_leads = ('I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6')
six_leads = ('I', 'II', 'III', 'aVR', 'aVL', 'aVF')
four_leads = ('I', 'II', 'III', 'V2')
three_leads = ('I', 'II', 'V2')
two_leads = ('I', 'II')
lead_sets = (twelve_leads, six_leads, four_leads, three_leads, two_leads)

test_data_directory = 'test_data'
test_header_files, test_recording_files = find_challenge_files(test_data_directory)
test_num_recordings = len(test_recording_files)
training_classes = list(labels)
test_classes = list(labels)
num_classes = len(classes)
num_classes

30

In [5]:
class My_Dataset_separate(Dataset) :
    def __init__(self,header_files, recording_files, leads, sample_length, sample_rate = 500) :
        super(Dataset,self).__init__()
        self.header_files = header_files
        self.recording_files = recording_files
        self.leads = leads
        self.sample_length = sample_length    
        self.sample_rate = sample_rate
    def __len__(self) :
         
        return len(self.recording_files)
    
    def __getitem__(self,index) :
        header = load_header(self.header_files[index])
        orig_sr = int(header.split(' ')[2])
        #print(orig_sr)
        recording = load_recording(self.recording_files[index])
        recordings = choose_leads(recording, header, self.leads)
        data = np.zeros((recordings.shape[0],self.sample_length))
        for i in range(len(recordings)):
            #print(type(data[i]))
            y = librosa.resample(recordings[i].astype(np.float), orig_sr, self.sample_rate, res_type='kaiser_best') 
            #print(y.shape[0])
            if y.shape[0] < self.sample_length :
                
                data[i,0:y.shape[0]] = y
            elif y.shape[0] >= self.sample_length:
                data[i] = y[0:self.sample_length]
        current_labels = get_labels(header)
        #print(current_labels)
        labels = np.zeros(( num_classes))
        for label in current_labels:
            if label in classes:
                j = classes.index(label)
                labels[j] = 1
#         data =recordings[:,0:self.sample_length]
       #data = data*10/np.linalg.norm(data)
#         for i in range(len(data)):
#             data[i] = data[i]/max(abs(data[i]))
#         orig_sr = int(header.split(' ')[2])
#         for i in range(len(data)):
#             #print(type(data[i]))
#             data[i] = librosa.resample(data[i].astype(np.float), orig_sr, self.sample_rate, res_type='kaiser_best')        
        recording_id = get_recording_id(header)
        if data.shape[1] < self.sample_length :
            print(data)
        return (data,labels,self.header_files[index])

In [6]:
sample_length = 4096
BATCH_SIZE  = 256
train_dataset = My_Dataset_separate(train_header_files, train_recording_files, twelve_leads,sample_length)
test_dataset = My_Dataset_separate(test_header_files, test_recording_files, twelve_leads,sample_length)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, num_workers=0, pin_memory=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1,
                             shuffle=False, num_workers=0, pin_memory=True)

In [7]:
a,b,c = next(iter(train_loader))
a.shape

torch.Size([256, 12, 4096])

In [8]:
import torch
import torch.nn as nn


class CNN1d(nn.Module):
    def __init__(self,in_channels):
        super(CNN1d, self).__init__()
        self.in_channels = in_channels
        self.conv1_1 = nn.Conv1d(in_channels=self.in_channels, out_channels=64, kernel_size=17, padding=8)
        self.conv1_2 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=17, padding=8)

        self.conv2_1 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=17, padding=8)
        self.conv2_2 = nn.Conv1d(in_channels=128, out_channels=128, kernel_size=3, padding=1)

        self.conv3_1 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=17, padding=8)
        self.conv3_2 = nn.Conv1d(in_channels=256, out_channels=256, kernel_size=17, padding=8)
        self.conv3_3 = nn.Conv1d(in_channels=256, out_channels=256, kernel_size=3, padding=8)

        self.conv4_1 = nn.Conv1d(in_channels=256, out_channels=512, kernel_size=17, padding=8)
        self.conv4_2 = nn.Conv1d(in_channels=512, out_channels=512, kernel_size=17, padding=8)
        self.conv4_3 = nn.Conv1d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

        self.conv5_1 = nn.Conv1d(in_channels=512, out_channels=512, kernel_size=17, padding=8)
        self.conv5_2 = nn.Conv1d(in_channels=512, out_channels=512, kernel_size=17, padding=8)
        self.conv5_3 = nn.Conv1d(in_channels=512, out_channels=512, kernel_size=17, padding=8)

        self.avgpool = nn.AdaptiveAvgPool1d(1)

        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 30)

    def forward(self, x):
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.avgpool(x)
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.avgpool(x)
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = F.relu(self.conv3_3(x))
        x = self.avgpool(x)
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))
        x = F.relu(self.conv4_3(x))
        x = self.avgpool(x)
        x = F.relu(self.conv5_1(x))
        x = F.relu(self.conv5_2(x))
        x = F.relu(self.conv5_3(x))
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.5) #dropout was included to combat overfitting
        x = F.relu(self.fc2(x))
        x = F.dropout(x, 0.5)
        x = F.sigmoid(self.fc3(x))
        return x


In [28]:
a = CNN1d(12)
output = a(torch.rand(24,12,4096))


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [29]:
output.shape


torch.Size([24, 30])

In [ ]:
num_epochs = 100
learning_rate = 0.001

model = CNN1d(12).cuda() 
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
Lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer , step_size = 20, gamma = .1)
label_directory = 'test_data'
output_directory = 'test_outputs'

for epoch in range(num_epochs):
        since = time.time()
        print('Epoch {}/{}'.format(epoch , num_epochs - 1))
        print('-' * 10)
    
        model.train() 
        running_loss = 0.0

        for i, (inputs, target, header_files) in tqdm.tqdm(enumerate(train_loader)):
            
            input_var = torch.autograd.Variable(inputs.cuda().float())
            target_var = torch.autograd.Variable(target.cuda().float())
            optimizer.zero_grad()

            # compute output
            output= model(input_var)
            # loss
            loss = criterion(output, target_var)
#             print(output)
#             print(target_var)
        
            if (i%5) == 0: 
                print('step: {} totalloss: {loss:.3f} '.format(i, loss = loss))

            loss.backward() 
            optimizer.step()  
            

            #print(loss.data.item())
            
            
            
            running_loss += loss.data.item()

                
            
        Lr_scheduler.step()
        epoch_loss = float(running_loss) / float(i)
        print(' Epoch over  Loss: {:.5f}'.format(epoch_loss))
        print('Testing_model............')
        auroc,auc = test_model(model, classes, test_loader, label_directory, output_directory)
        PATH = f'/content/drive/MyDrive/Thesis/model checkpoint classifiers/CNN1d-{epoch}-aur-{round(auroc,3)}-auc {round(auc,3)} .pth' 
        torch.save(model.state_dict(), PATH)

Epoch 0/99
----------


0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
1it [00:06,  6.57s/it]

step: 0 totalloss: 0.692 


6it [00:30,  5.34s/it]

step: 5 totalloss: 0.675 


...

46it [03:10,  4.46s/it]

step: 45 totalloss: 0.184 


51it [03:28,  4.45s/it]

step: 50 totalloss: 0.157 


52it [03:32,  4.09s/it]